# (목적)Gradient를 확인한다
## 전체 Gradient에 3을 곱한것과 parameter 별로 gradient에 3을 곱한 것이 차이가 나는지 확인

In [55]:
import torch
import torch.nn as nn

import torch.nn.functional as F

# 예제 데이터 생성
X = torch.randn(100, 2)
y = torch.randint(0, 2, (100,))

# 모델 정의
class ClassificationModel(nn.Module):
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 2)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

model = ClassificationModel()

# Cross Entropy Loss와 KL Divergence Loss 정의
cross_entropy_loss = nn.CrossEntropyLoss()
kl_divergence_loss = nn.KLDivLoss(reduction='batchmean')

# Forward Pass
outputs = model(X)

# Cross Entropy Loss 계산
ce_loss = F.cross_entropy(outputs, y)

# KL Divergence Loss 계산
p = torch.tensor([0.1, 0.2])  # 예시 분포
kl_loss = kl_divergence_loss(torch.log(outputs), p)
# kl_loss = ce_loss

# Gradient 계산 (torch.autograd.grad 사용)
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
kl_gradient = torch.autograd.grad(kl_loss, model.parameters(), retain_graph=True)

# Gradient 확인
print("Cross Entropy Loss Gradient:")
for grad_param in ce_gradient:
    print(grad_param)

# print("\nKL Divergence Loss Gradient:")
# for grad_param in kl_gradient:
#     print(grad_param)

Cross Entropy Loss Gradient:
tensor([[-0.0106, -0.0197],
        [-0.0503, -0.0939]])
tensor([-0.0105, -0.0500])
tensor([[-0.0456,  0.0535],
        [ 0.0456, -0.0535]])
tensor([-0.0711,  0.0711])


In [56]:
ce_loss *= 2
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)

# Gradient 확인
print("Cross Entropy Loss Gradient:")
for grad_param in ce_gradient:
    print(grad_param)

Cross Entropy Loss Gradient:
tensor([[-0.0211, -0.0394],
        [-0.1007, -0.1877]])
tensor([-0.0210, -0.1000])
tensor([[-0.0912,  0.1069],
        [ 0.0912, -0.1069]])
tensor([-0.1422,  0.1422])


In [57]:
kl_loss

tensor(nan, grad_fn=<DivBackward0>)

In [58]:
total_loss = 1 * ce_loss +  2* kl_loss
total_loss
total_gradient = torch.autograd.grad(total_loss, model.parameters(), retain_graph=True)
print(total_gradient)

(tensor([[ 0.2023, -0.0481],
        [ 0.6727, -0.4599]]), tensor([0.3982, 1.4024]), tensor([[-0.5114,  0.4607],
        [ 0.4840, -0.3643]]), tensor([ 1.4619, -2.1532]))


In [59]:
names_grads_copy = {}

for param_name, ce_grad, kl_grad in zip(model.parameters(), ce_gradient, kl_gradient):
    
    #kl_grad = None
    
    if not kl_grad == None:
        names_grads_copy[param_name] = torch.tensor(1) * ce_grad + torch.tensor(2) *  kl_grad
    else:
        names_grads_copy[param_name] = torch.tensor(1) * ce_grad 
    
for key, value in names_grads_copy.items():
    print(value)

tensor([[ 0.2023, -0.0481],
        [ 0.6728, -0.4599]])
tensor([0.3982, 1.4024])
tensor([[-0.5114,  0.4607],
        [ 0.4840, -0.3643]])
tensor([ 1.4619, -2.1532])


## 1. Gradient를 추가적으로 구해도 retain_graph=True를 설정하면, 값이 변하지 않는다

In [60]:
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

tensor([[-0.0211, -0.0394],
        [-0.1007, -0.1877]])
tensor([-0.0210, -0.1000])
tensor([[-0.0912,  0.1069],
        [ 0.0912, -0.1069]])
tensor([-0.1422,  0.1422])


# 2. 전체 gradient에 3을 곱한다
## 1) 안된다

In [61]:
ce_gradient =  torch.tensor(3) * torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

tensor([[-0.0211, -0.0394],
        [-0.1007, -0.1877]])
tensor([-0.0210, -0.1000])
tensor([[-0.0912,  0.1069],
        [ 0.0912, -0.1069]])
tensor([-0.1422,  0.1422])
tensor([[-0.0211, -0.0394],
        [-0.1007, -0.1877]])
tensor([-0.0210, -0.1000])
tensor([[-0.0912,  0.1069],
        [ 0.0912, -0.1069]])
tensor([-0.1422,  0.1422])
tensor([[-0.0211, -0.0394],
        [-0.1007, -0.1877]])
tensor([-0.0210, -0.1000])
tensor([[-0.0912,  0.1069],
        [ 0.0912, -0.1069]])
tensor([-0.1422,  0.1422])


## 2) 덧셈 조차 하면 안된다

In [62]:
total_gradient = ce_gradient + kl_gradient
total_gradient

(tensor([[-0.0211, -0.0394],
         [-0.1007, -0.1877]]),
 tensor([-0.0210, -0.1000]),
 tensor([[-0.0912,  0.1069],
         [ 0.0912, -0.1069]]),
 tensor([-0.1422,  0.1422]),
 tensor([[-0.0211, -0.0394],
         [-0.1007, -0.1877]]),
 tensor([-0.0210, -0.1000]),
 tensor([[-0.0912,  0.1069],
         [ 0.0912, -0.1069]]),
 tensor([-0.1422,  0.1422]),
 tensor([[-0.0211, -0.0394],
         [-0.1007, -0.1877]]),
 tensor([-0.0210, -0.1000]),
 tensor([[-0.0912,  0.1069],
         [ 0.0912, -0.1069]]),
 tensor([-0.1422,  0.1422]),
 tensor([[ 0.1117, -0.0044],
         [ 0.3867, -0.1361]]),
 tensor([0.2096, 0.7512]),
 tensor([[-0.2101,  0.1769],
         [ 0.1964, -0.1287]]),
 tensor([ 0.8020, -1.1477]))

## 3) loss에 3을 곱해줘야 내가 원하는 값을 얻을 수 있다

In [63]:
ce_gradient =  torch.autograd.grad(torch.tensor(3) * ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

tensor([[-0.0634, -0.1181],
        [-0.3020, -0.5632]])
tensor([-0.0629, -0.2999])
tensor([[-0.2736,  0.3208],
        [ 0.2736, -0.3208]])
tensor([-0.4267,  0.4267])


# 3. 각 parameter 별로 3을 곱해준다

In [64]:
# Gradient 확인
print("Cross Entropy Loss Gradient:")
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(torch.tensor(3) * grad_param)

Cross Entropy Loss Gradient:
tensor([[-0.0634, -0.1181],
        [-0.3020, -0.5632]])
tensor([-0.0629, -0.2999])
tensor([[-0.2736,  0.3208],
        [ 0.2736, -0.3208]])
tensor([-0.4267,  0.4267])


## 4. weight에 2를 곱하고 gradient를 구하면 값이 달라진다

In [65]:
# 각 레이어의 가중치에 2를 곱함
for param in model.parameters():
    param.data = param.data * 2

ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

tensor([[-0.0211, -0.0394],
        [-0.1007, -0.1877]])
tensor([-0.0210, -0.1000])
tensor([[-0.0912,  0.1069],
        [ 0.0912, -0.1069]])
tensor([-0.1422,  0.1422])


# 5. 각 parameter 별로 3을 더해준다

In [66]:
# Gradient 확인
print("Cross Entropy Loss Gradient:")
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(grad_param)

Cross Entropy Loss Gradient:
tensor([[-0.0211, -0.0394],
        [-0.1007, -0.1877]])
tensor([-0.0210, -0.1000])
tensor([[-0.0912,  0.1069],
        [ 0.0912, -0.1069]])
tensor([-0.1422,  0.1422])


In [67]:
# Gradient 확인
print("Cross Entropy Loss Gradient:")
ce_gradient = torch.autograd.grad(ce_loss, model.parameters(), retain_graph=True)
for grad_param in ce_gradient:
    print(torch.tensor(3) + grad_param)

Cross Entropy Loss Gradient:
tensor([[2.9789, 2.9606],
        [2.8993, 2.8123]])
tensor([2.9790, 2.9000])
tensor([[2.9088, 3.1069],
        [3.0912, 2.8931]])
tensor([2.8578, 3.1422])
